In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [4]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Define_Basic_Parameters

In [5]:
# define the num which represent the true number of verified sample-point sets
num = 5

# define the tree_num used as key points that record the change of accuracy
tree_num = [1] + list(range(10,121,10))

# define the year range
year_start = [f'{i}-01-01' for i in range(1987,2020,3)]
year_end   = [f'{i}-12-31' for i in range(1989,2020,3)]

year_range = list(zip(year_start,year_end))[-num:][::-1]

year_name_all  = list([f'{span[0]}_{span[1]}' for span in zip(range(1987,2020,3),range(1989,2020,3)) ])
year_name = year_name_all[-num:][::-1]

In [6]:
year_range

[('2017-01-01', '2019-12-31'),
 ('2014-01-01', '2016-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2005-01-01', '2007-12-31')]

In [7]:
year_name

['2017_2019', '2014_2016', '2011_2013', '2008_2010', '2005_2007']

##### Classify Sample_with_img_value

In [8]:
path = 'users/wangjinzhulala/North_China_Plain_Python/Sample_with_Landsat_Fourier_Nomalized_img_value'

Sample_with_value = [ee.FeatureCollection(f'{path}/Sample_{year}_extract_Landsat_Fourier_Normalized_img')
                    for year in year_name]

In [50]:
# Initiate the accuracy dictionary
Combo_instance = {}

for fe,year in  zip(Sample_with_value,year_name):
    
    # _________________________________prepare the band names_________________________________
    
    # use the first element to get all band name
    bands = list(fe.first().getInfo()['properties'].keys())
    
    # get the Landsat_band
    Landsat_re   = re.compile(r'^B\d')
    Landsat_band = list(filter(Landsat_re.match,bands))

    # get the Fourier_band
    Fourier_re   = re.compile(r'^EVI|NDBI|NDVI')
    Fourier_band = list(filter(Fourier_re.match,bands))
    
    # get the Mean_Normalized band
    Mean_re         = re.compile(r'^Mean')
    Mean_Normalized = list(filter(Mean_re.match,bands))
    
    # ________________________Create sample classification instaces_____________________________
    
    # create band_combinations
    band_combination = [Landsat_band,
                        Fourier_band,
                        Landsat_band + Fourier_band,
                        Landsat_band + Mean_Normalized,
                        Fourier_band + Mean_Normalized,
                        Landsat_band + Fourier_band + Mean_Normalized]
    
    # create comno_names
    combination_name = ['Landsat',
                        'Fourier',
                        'Landsat_Fourier',
                        'Landsat_Mean',
                        'Fourier_Mean',
                        'Landsat_Fourier_Mean']
    
    # test the accuracy of each combo
    for combo,name in zip(band_combination,combination_name):
               
        for tree in tree_num:
            
            # Instatiate the class with a name
            Accuracy_assess = Classification( year_name      = year,
                                              Input_img      = 0,
                                              input_band     = combo,
                                              Verified_point = fe,
                                              Tree_num       = tree)
            
            # add accuracy to the dict
            Accuracy_assess.Stp_2_Classification_on_Samples()
            
            # get the accuracy value
            Combo_instance[(year,name,tree)] = Accuracy_assess.Test_sample_classification

In [51]:
# innitialize the accuracy dict
Combo_acc = {}

# innitilize operation flag to report the process
Opts_flat  = 0
Opts_tatal = len(year_name) * len(combination_name) * len(tree_num)

# unpack the accuracy_instances
for year in year_name:
    for name in combination_name:
        for tree in tree_num:
            
            Opts_flat+=1
            
            Combo_acc[(year,name,tree)] = Accuracy_assesment(Combo_instance[(year,name,tree)])\
                                          .Stp_1_Calculate_Accuracy()
            # print out the process
            print(f'{year}_{name}_{str(tree).zfill(3)} accuracy computation finished! ==>{Opts_flat}/{Opts_tatal}')
        print('_________________')

EEException: Classifier.train: No input properties found.